In [ ]:
import param_estimator
import pickle
import estimator
import matplotlib.pyplot as plt
import logging
import numpy as np

plt.style.use("dark_background")
logging.basicConfig(level=logging.INFO)
logging.getLogger("activation_learner").setLevel(logging.DEBUG)

In [ ]:
import librosa

librosa.filters.mel(sr=22050, n_fft=256).dtype

In [ ]:
with open("model.pickle", "rb") as f:
    model = pickle.load(f)

In [ ]:
volumes = param_estimator.volumes(model.nmf.H.toarray(), model.colsum, model.split_idx)
positions = param_estimator.positions(model.nmf.H.toarray(), model.split_idx)
param_estimator.plot_vol_pos(volumes, positions, model.boundaries, model.fs)

In [ ]:
import scipy.signal

SILENCE_THRESHOLD = 0.01
PLAYING_THRESHOLD = 0.99
SMOOTH_WIN_S = 1.0
mix_times = param_estimator.frames_to_times(
    np.arange(len(model.boundaries[-1])), model.boundaries[-1], model.fs
)


def largest_unmasked_clump(masked_array):
    slices = np.ma.clump_unmasked(masked_array)
    assert len(slices) > 0
    return max(slices, key=lambda s: s.stop - s.start)


def largest_masked_clump(masked_array):
    slices = np.ma.clump_masked(masked_array)
    assert len(slices) > 0
    return max(slices, key=lambda s: s.stop - s.start)


for pos, vol, bounds in zip(positions, volumes, model.boundaries):
    # convert frame to times
    pos = param_estimator.frames_to_times(pos, bounds, model.fs)
    times = param_estimator.frames_to_times(np.arange(len(pos)), bounds, model.fs)

    vol_filt = scipy.signal.medfilt(vol, 15)
    min_vol, max_vol = np.min(vol_filt), np.max(vol_filt)
    silence_threshold, playing_threshold = 0.1, max_vol * 0.9
    threshold = (max_vol - min_vol) / 2
    crossings = np.where(np.diff(np.sign(vol_filt - threshold)))[0]
    fades = []
    for crossing in crossings:
        rising = vol_filt[crossing] < vol_filt[crossing + 1]
        if rising:
            fade_start = crossing + np.argmax(vol[crossing:-1] >= playing_threshold)
            fade_end = np.argmax(vol[:crossing] <= silence_threshold)
        else:
            fade_start = np.argmax(vol[:crossing:-1] >= playing_threshold)
            fade_end = crossing + np.argmax(vol[crossing:] <= silence_threshold)
        fades.append((fade_start, fade_end))
    plt.figure(figsize=(15, 5))
    plt.plot(vol)
    plt.plot(vol_filt)
    for i in crossings:
        plt.axvline(i, color="red")
    for a, b in fades:
        print(a, b)
        plt.axvline(a, color="blue")
        plt.axvline(b, color="yellow")
    plt.axhline(silence_threshold)
    plt.axhline(playing_threshold)

    plt.legend()
    plt.show()

In [ ]:
mix_times = param_estimator.frames_to_times(
    np.arange(len(model.boundaries[-1])), model.boundaries[-1], model.fs
)
for pos, vol, bounds in zip(positions, volumes, model.boundaries):
    # convert frame to times
    pos = param_estimator.frames_to_times(pos, bounds, model.fs)
    times = param_estimator.frames_to_times(np.arange(len(pos)), bounds, model.fs)

    vol_filt = scipy.signal.medfilt(vol, 15)

    min_vol, max_vol = np.min(vol_filt), np.max(vol_filt)
    silence_threshold, playing_threshold = 0.1, max_vol * 0.9
    threshold = (max_vol - min_vol) / 2

    plt.plot(mix_times, vol_filt)
    plt.plot(mix_times, np.gradient(vol_filt))
    plt.plot(mix_times, np.sign(vol_filt - threshold))
    plt.show()